# Overview



# Load Dependencies



In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.2f}'.format

# Import Stock Prices

**Halliburton - HAL**<BR>
> an American multinational corporation<BR>
> one of the world's largest oil field service companies<BR>
> operations in more than 70 countries<BR>


In [ ]:
df_HAL = pd.read_csv(filepath_or_buffer='../Data/HAL.csv', 
                     parse_dates=True,
                     index_col='Date')

df_HAL.head()


# Import a basket of Indices & Futures

**Index - SP500**<BR>
> a stock market index tracking the performance of 500 large companies listed on stock exchanges in the United States. <BR>
> one of the most commonly followed equity indices<BR>
> as of December 31, 2020, more than $5.4 trillion was invested in assets tied to the performance of the index


**Commodities Futures - OIl, Gold, Natural Gas**
> contracts between buyers and sellers that trade on exchanges<BR>
> Oil Futures<BR>
> Gold Futures<BR>
> Natural Gas Futures<BR>



In [ ]:
df_basket = pd.read_csv(filepath_or_buffer='../Data/basket.csv',
                       parse_dates=True,
                       index_col='Date')

df_basket.head()

# Concatenate HAL and Basket 

For convenience, concatenate HAL and derivs into a single DataFrame

In [ ]:
df = pd.concat(objs = [df_HAL, df_basket], axis = 1)

df.head()

# Correlate returns of HAL against Index and Futures

Use the `pct_change()` Dataframe method


In [ ]:
df = df.pct_change()

df.head()

In [ ]:
df.corr()

# Produce a Heatmap

## Simple Heatmap

In [ ]:
plt.figure(figsize=(16, 6))

# define the mask to set the values in the upper triangle to True
sns.heatmap(df.corr(), annot=True);


## Triangle Correlation Heatmap

In [ ]:
plt.figure(figsize=(16, 6))

# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df.corr(), dtype=np.bool))

heatmap = sns.heatmap(df.corr(), mask=mask, annot=True)

# Regression Analysis

Regression is an econometric method that allows a user to attempt to see what variables drive another variable.  Regression analysis calculates a number of important values

**Beta, Significance & P-Values**

A **beta** is what a 1-unit increase in the specific variable does to the dependent variable.
For example a .5 beta means that if a variable is increased by 1, the dependent variable would be expected to go up by 0.5. 

**Significance** measures the probability that a particular variable's value was a random occurrence.
For example, a significance factor of 0.05 means there was a 5% chance that this was a random event, 0.01 means a 1% chance, and so on. 

A **p-value** is used to provide the smallest level of significance at which the null hypothesis would be rejected.

Beta, Significance and p-values are important because they are indicators to users about what is significant.

For example, we might expect that gold prices effect all stocks. However, when we control for the market (by using SPY) we notice that the reason gold is correlated with all these stocks is because it is also correlated with the market! This gives us better precision since we see which firms really get effected by gold versus which just get effected by the overall market environment.

A very accurate regression analysis is performed using the statsmodels OLS function (ordinary least squares). The formula parameter sets the dependent variable and the matrix of factors. <br><br>
> `HAL` **~** `SP500 + Oil + Gold + Gas`<br>

In the above expression<br>
> `HAL` is the dependent variable <br>
> `SP500 + Oil + Gold + Gas` are the set of features in the model.


In [ ]:
# Create a regression model
import statsmodels.formula.api as smf

model = smf.ols(formula="HAL ~ SP500 + Oil + Gold + Gas", data=df)

# fit the model
result = model.fit()

# Prouduce the regression report
result.summary()


In [ ]:
print(result.summary())

In [ ]:
# display all p-values
result.pvalues 

In [ ]:
# display all p-values < 0.05
result.pvalues < .05


In [ ]:
# display the `features` less than 0.05
is_significant = result.pvalues < .05
result.params[is_significant]

**Note the following**<br>

1) The values below will change when the dates change
1) 2 factors have a p-value  (**P>|t|**) < 0.05 : SP500 and Oil<br>
Given that PXD is an oil exploration company, it's daily returns follow very closely the returns of both the SP500 index and the price of Oil.

2) The HAL beta for stock market returns is relatively high at approx 0.95, and Oil has a beta of approx 0.75. This means that for every 1% change in the price of oil, users can expect a 0.75% change in the price of the HAL stock. 

3) Another way of thinking of this is 
> HAL = `(0.95 X SPY) + (0.75X Oil)`

We can turn then above statements into a function.

In [ ]:
def regress(ticker, df, p_threshold=0.05):
    
    formula = f'{ticker} ~ SP500 + Oil + Gold + Gas'
    
    fitted = smf.ols(data=df, formula=formula).fit()
    
    return fitted.params[fitted.pvalues < p_threshold].rename(ticker)



## Extract Data from the report

Use the `regress()` function to
- display the report
- display only the p-values
- display only the p-values less than 0.05

And most importantly, use the params attribute of the OLS report to display the `features` less than 0.05



In [ ]:
# run the regression
result = regress("HAL", df)

# Create a DataFrame
df_regress = pd.DataFrame(data = result)

df_regress

# Perform this analysis for any abritrary stock

Rather than importing a single stock data, we can easily import all stock data for all companies in the SP500 index. 
We will follow the same “pattern”:
- Load a DataFrame that represents the basket.
- Load a DataFrame of all SP500 stocks.
- Calculate the daily returns and concatenate both DataFrames.
- Calculate the Betas for ALL stocks when regressions are performed against the basket.
- Save the results to a Spreadsheet.

The first 3 steps are:

In [ ]:
# Load in the Stock Prices
df_stocks = pd.read_csv(filepath_or_buffer = '../Data/prices.csv',
                        parse_dates=True,
                        index_col='Date').pct_change()

# Load in the basket
df_basket = pd.read_csv(filepath_or_buffer='../Data/basket.csv',
                        parse_dates=True,
                        index_col='Date')


df_returns = df_stocks.join(df_basket.pct_change(), how='inner')

df_returns.shape

## Calculate Betas for ALL stocks with p-value < 0.05


A very simple loop achieves this.<br>

**Note**<br>
1) the slice `f_returns.columns[:-4]` returns all BUT the last 4 columns as the last 4 columns are our basket.

2) We are progressively appending the results of the regression into a single list with each element of the list containing a series of results


## Using a simple loop

For the more empirical programmer

In [ ]:
results = []
for tick in df_returns.columns[:-4]:
    results.append(regress(ticker=tick, df=df_returns))
    

## Using a comprehension

An alternate way to create a list of results, for the pythonistas who like one-liners

In [ ]:
results = [regress(ticker=tick, df=df_returns) for tick in df_returns.columns[:-4]]



## Display a few rows

First and last

In [ ]:
display(results[0])
display(results[len(results)-1])


# Create a Dataframe from the results

In [ ]:
df_all_Betas = pd.DataFrame(data=results)

df_all_Betas.head()

## Export the DataFrame to a Spreadsheet

Using the python resource manager syntax


In [ ]:
# Create a writer
with pd.ExcelWriter('../Output/BasketBetas.xlsx') as writer:
    
    # All Betas
    df_all_Betas.to_excel(writer, sheet_name='Betas')
    
    # Transposed
    df_all_Betas.transpose().to_excel(writer, sheet_name='Tposed')
    
    # Sorted by SP500 with Nulls filtered out
    df_out = df_all_Betas[df_all_Betas['SP500'].notnull()].sort_values(by='SP500', ascending=False)
    df_out.to_excel(writer, sheet_name='SP500')
    
    # Sorted by Oil with Nulls filtered out
    df_out = df_all_Betas[df_all_Betas['Oil'].notnull()].sort_values(by='Oil', ascending=False)
    df_out.to_excel(writer, sheet_name='Oil')
    
    # Sorted by Oil with Nulls filtered out
    df_out = df_all_Betas[df_all_Betas['Gold'].notnull()].sort_values(by='Gold', ascending=False)
    df_out.to_excel(writer, sheet_name='Gold')
    
    # Sorted by Oil with Nulls filtered out
    df_out = df_all_Betas[df_all_Betas['Gas'].notnull()].sort_values(by='Gas', ascending=False)
    df_out.to_excel(writer, sheet_name='Gas')
    

In [ ]:
df_all_Betas[df_all_Betas['SP500'].notnull()].sort_values(by='SP500', ascending=False).head()

In [ ]:
df_all_Betas[df_all_Betas['Oil'].notnull()].sort_values(by='Oil', ascending=False).head()

In [ ]:
df_all_Betas[df_all_Betas['Gold'].notnull()].sort_values(by='Gold', ascending=False).head()

In [ ]:
df_all_Betas[df_all_Betas['Gold'].notnull()].sort_values(by='Gold', ascending=True).head()

In [ ]:
df_all_Betas[df_all_Betas['Gas'].notnull()].sort_values(by='Gas', ascending=False).head()